# Setup

In [ ]:
# Load required packages
library(tidyverse)
library(bedtoolsr)
library(ggVennDiagram)

# Analysis of selective sweeps 

## XP-nSL

- I'll start by looking at genome-wide XP-xSL values, estimated in 50 kb windows
- In each window, I estimated the mean XP-nSL score and the proportion of scores that are either above 2 or less than -2.
- Positive outlier windows are those that are in the top 1% of the genome-wide mean XP-nSL distribution AND the top 1% of distribution of scores > 2. These windows represent positive selection in urban habitats
- Negative outlier windows are those that are in the bottom 1% of the genome-wide XP-xSL distribution AND the top 1% of distribution of scores < -2. These windows represent positive selection in rural habitats.
- I'll look at the top 10 windows under selection for urban and rural habitats separately.
- I'll merge outlier windows within 50Kb of one another into larger outlier regions

In [ ]:
# Load observed XP-nSL data
obs_xpnsl_df <- read_delim(snakemake@input[["win_xpnsl"]])
head(obs_xpnsl_df)

In [ ]:
# Distribution of the number of sites per window
xpnsl_nsites_thresh <- 50
xpnsl_nSites_hist <- obs_xpnsl_df %>% 
    ggplot(aes(x = n)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of windows") + xlab("Number of sites") +
        geom_vline(xintercept = xpnsl_nsites_thresh, color = "red", linetype = "dashed", linewidth = 1) +
        theme_classic() +
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 17))

xpnsl_nSites_hist
ggsave(filename = snakemake@output[["xpnsl_nSites_hist"]], plot = xpnsl_nSites_hist, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")

### Filter and identify outlier windows

In [ ]:
# Function to assign outliers to XP-nSL windows
assign_xpnsl_outliers <- function(df){

    df_filt <- df %>%
        filter(n >= xpnsl_nsites_thresh)

    # Get critical values for mean XP-nSL score and proportions greater or lesser than 2 and -2, respectively
    xpnsl_score_quant <- quantile(df_filt %>% pull(mean), probs = c(0.01, 0.99))
    xpnsl_gtprop_quant <- quantile(df_filt %>% pull(gt_frac), probs = 0.99)
    xpnsl_ltprop_quant <- quantile(df_filt %>% pull(lt_frac), probs = 0.99)

    # Identify outliers and add as categorical variable to windows dataframe
    df_filt <- df_filt %>%
        mutate(xpnsl_score_outlier = ifelse(mean <= xpnsl_score_quant[1] | mean >= xpnsl_score_quant[2], 1, 0),
               xpnsl_gtprop_outlier = ifelse(gt_frac >= xpnsl_gtprop_quant, 1, 0),
               xpnsl_ltprop_outlier = ifelse(lt_frac >= xpnsl_ltprop_quant, 1, 0),
               direction = case_when(xpnsl_score_outlier == 1 & mean > 0 & xpnsl_gtprop_outlier == 1 ~ 'Urban',
                                     xpnsl_score_outlier == 1 & mean < 0 & xpnsl_ltprop_outlier == 1 ~ 'Rural',
                                     TRUE ~ 'None')) %>% 
    mutate(prop_outlier = case_when(direction == 'Urban' ~ gt_frac,
                                    direction == 'Rural' ~ lt_frac,
                                    TRUE ~ NA))

    return(df_filt)
}

obs_xpnsl_df_filt <- assign_xpnsl_outliers(obs_xpnsl_df)

sprintf("%s of %s XP-nSL windows remaining after removing those with less than %s sites", nrow(obs_xpnsl_df_filt), nrow(obs_xpnsl_df), xpnsl_nsites_thresh)

head(obs_xpnsl_df_filt)

### Generate Manhattan plot

In [ ]:
# Setting up cummulative genome-wide x-axis
data_cum <- obs_xpnsl_df_filt %>% 
    group_by(Chr) %>% 
    summarise(max_winCenter = max(winCenter)) %>% 
    mutate(winCenter_add = lag(cumsum(max_winCenter), default = 0)) %>% 
    dplyr::select(Chr, winCenter_add)

obs_xpnsl_df_filt_mod <- obs_xpnsl_df_filt %>%
    inner_join(data_cum, by = "Chr") %>% 
    mutate(winCenter_cum = winCenter + winCenter_add) 

axis_set <- obs_xpnsl_df_filt_mod %>% 
  group_by(Chr) %>% 
  summarize(center = mean(winCenter_cum))

# Get XP-nSL outliers for plotting as separate layers
xpnsl_not_outlier <- obs_xpnsl_df_filt_mod %>% filter(direction == 'None')
urban_xpnsl_outliers <- obs_xpnsl_df_filt_mod %>% filter(direction == 'Urban')
rural_xpnsl_outliers <- obs_xpnsl_df_filt_mod %>% filter(direction == 'Rural')

# Get XP-nSL score quantile for plotting
xpnsl_score_quant <- quantile(obs_xpnsl_df_filt_mod %>% pull(mean), probs = c(0.01, 0.99))

xpnsl_not_outlier <- xpnsl_not_outlier %>%
    mutate(chrom_cat = case_when(Chr == 'Chr01_Occ' ~ 'One',
                                 Chr == 'Chr01_Pall' ~ 'Two',
                                 Chr == 'Chr02_Occ' ~ 'One',
                                 Chr == 'Chr02_Pall' ~ 'Two',
                                 Chr == 'Chr03_Occ' ~ 'One',
                                 Chr == 'Chr03_Pall' ~ 'Two',
                                 Chr == 'Chr04_Occ' ~ 'One',
                                 Chr == 'Chr04_Pall' ~ 'Two',
                                 Chr == 'Chr05_Occ' ~ 'One',
                                 Chr == 'Chr05_Pall' ~ 'Two',
                                 Chr == 'Chr06_Occ' ~ 'One',
                                 Chr == 'Chr06_Pall' ~ 'Two',
                                 Chr == 'Chr07_Occ' ~ 'One',
                                 Chr == 'Chr07_Pall' ~ 'Two',
                                 Chr == 'Chr08_Occ' ~ 'One',
                                 Chr == 'Chr08_Pall' ~ 'Two'))

xpnsl_manhat <- ggplot() +
        geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.4, size = 3, 
                   aes(x = winCenter_cum, y = mean, fill = chrom_cat, color = chrom_cat)) +
        geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 1, size = 3, color = '#003876', fill = '#003876',
                   aes(x = winCenter_cum, y = mean)) +
        geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 1, size = 3, color = '#007243', fill = '#007243',
                   aes(x = winCenter_cum, y = mean)) +
        geom_hline(yintercept = xpnsl_score_quant, color = "grey40", linetype = "dashed") +
        scale_x_continuous(label = axis_set$Chr, breaks = axis_set$center) +
        scale_y_continuous(expand = c(0,0)) +
        coord_cartesian(ylim = c(-6, 5)) +
        scale_fill_manual(values = c("black", "grey40")) + 
        scale_color_manual(values = c("black", "grey40")) + 
        ylab('Normalized XP-nSL') + xlab('Chromosomes') +
        ggtitle('Urban-Rural XP-nSL') +
        theme_classic() +
        theme(
            legend.position = "none",
            panel.border = element_blank(),
            panel.grid.major.x = element_blank(),
            panel.grid.minor.x = element_blank(),
            axis.text = element_text(size=16),
            axis.title = element_text(size=20),
            axis.text.x = element_text(angle = 45, hjust = 1),
            plot.title = element_text(size=20, face="bold")
          )

options(repr.plot.width = 20, repr.plot.height = 6)
xpnsl_manhat
ggsave(filename = snakemake@output[["xpnsl_manhat"]], plot = xpnsl_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
load_hapstat_norm <- function(path, stat){
    chrom_name <- str_extract(basename(path), pattern = "Chr\\d+_((Occ)|(Pall))")
    habitat <- str_extract(basename(path), pattern = "(Urban|Rural)")

    if (stat == "ihh12"){
        df <- suppressMessages(read_delim(path, delim = "\t")) %>% 
            mutate(Chr = chrom_name, habitat = habitat)
    } else if(stat == "xpnsl"){
        df <- suppressMessages(read_delim(path, delim = "\t")) %>% 
            mutate(Chr = chrom_name)
    } else if (stat == "ihs"){
        cols <- c("id", "pos", "1_freq", "ihh1", "ihh0", "iHs", "normihs", "crit")
        df <- suppressMessages(read_delim(path, delim = "\t", col_names = cols)) %>% 
            mutate(Chr = chrom_name, habitat = habitat)
    } else if (stat == "nsl"){
        cols <- c("id", "pos", "1_freq", "sl1", "sl0", "nSL", "normnsl", "crit")
        df <- suppressMessages(read_delim(path, delim = "\t", col_names = cols)) %>% 
            mutate(Chr = chrom_name, habitat = habitat)
    }
    return(df)
}

In [ ]:
xpnsl_persite <- snakemake@input[["norm_xpnsl"]] %>% 
    purrr::map_dfr(., load_hapstat_norm, stat = "xpnsl") %>% 
    rename("normxpnsl" = "normxpehh") %>% 
    mutate(crit = ifelse(normxpnsl < -2 | normxpnsl > 2, 1, 0))

In [ ]:
# Setting up cummulative genome-wide x-axis
data_cum <- xpnsl_persite %>% 
    group_by(Chr) %>% 
    summarise(max_pos = max(pos)) %>% 
    mutate(pos_add = lag(cumsum(max_pos), default = 0)) %>% 
    dplyr::select(Chr, pos_add)

xpnsl_persite_mod <- xpnsl_persite %>%
    inner_join(data_cum, by = "Chr") %>% 
    mutate(pos_cum = pos + pos_add) 

axis_set <- xpnsl_persite_mod %>% 
  group_by(Chr) %>% 
  summarize(center = mean(pos_cum))

# Get XP-nSL outliers for plotting as separate layers
xpnsl_not_outlier <- xpnsl_persite_mod %>% filter(crit == 0)
urban_xpnsl_outliers <- xpnsl_persite_mod %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- xpnsl_persite_mod %>% filter(crit == 1 & normxpnsl < 0)

xpnsl_not_outlier <- xpnsl_not_outlier %>%
    mutate(chrom_cat = case_when(Chr == 'Chr01_Occ' ~ 'One',
                                 Chr == 'Chr01_Pall' ~ 'Two',
                                 Chr == 'Chr02_Occ' ~ 'One',
                                 Chr == 'Chr02_Pall' ~ 'Two',
                                 Chr == 'Chr03_Occ' ~ 'One',
                                 Chr == 'Chr03_Pall' ~ 'Two',
                                 Chr == 'Chr04_Occ' ~ 'One',
                                 Chr == 'Chr04_Pall' ~ 'Two',
                                 Chr == 'Chr05_Occ' ~ 'One',
                                 Chr == 'Chr05_Pall' ~ 'Two',
                                 Chr == 'Chr06_Occ' ~ 'One',
                                 Chr == 'Chr06_Pall' ~ 'Two',
                                 Chr == 'Chr07_Occ' ~ 'One',
                                 Chr == 'Chr07_Pall' ~ 'Two',
                                 Chr == 'Chr08_Occ' ~ 'One',
                                 Chr == 'Chr08_Pall' ~ 'Two'))

persite_xpnsl_manhat <- ggplot() +
        geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.1, size = 0.25, 
                   aes(x = pos_cum, y = normxpnsl, fill = chrom_cat, color = chrom_cat)) +
        geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.1, size = 0.25, color = '#003876', fill = '#003876',
                   aes(x = pos_cum, y = normxpnsl)) +
        geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.1, size = 0.25, color = '#007243', fill = '#007243',
                   aes(x = pos_cum, y = normxpnsl)) +
        geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
        geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
        scale_x_continuous(label = axis_set$Chr, breaks = axis_set$center) +
        scale_y_continuous(expand = c(0,0), breaks = seq(-8, 6, 2)) +
        coord_cartesian(ylim = c(-9, 7)) +
        scale_fill_manual(values = c("black", "grey40")) + 
        scale_color_manual(values = c("black", "grey40")) + 
        ylab('Normalized XP-nSL') + xlab('Chromosomes') +
        ggtitle('Per-Site Urban-Rural XP-nSL') +
        theme_classic() +
        theme(
            legend.position = "none",
            panel.border = element_blank(),
            panel.grid.major.x = element_blank(),
            panel.grid.minor.x = element_blank(),
            axis.text = element_text(size=16),
            axis.title = element_text(size=20),
            axis.text.x = element_text(angle = 45, hjust = 1),
            plot.title = element_text(size=20, face="bold")
          )

options(repr.plot.width = 20, repr.plot.height = 6)
persite_xpnsl_manhat
ggsave(filename = snakemake@output[["xpnsl_manhat_persite"]], plot = persite_xpnsl_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["xpnsl_manhat_persite_png"]], plot = persite_xpnsl_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

### Add ranks and percentiles from permuted distributions

#### Add ranks

In [ ]:
# Function to add ranks to XP-nSL windows based on the proportion of outlier scores
add_xpnsl_ranks <- function(df){
        
    df_out <- df %>% 
        arrange(desc(prop_outlier)) %>% 
        mutate(xpnsl_rank = 1:n())
    return(df_out)
}

obs_xpnsl_df_filt <- obs_xpnsl_df_filt %>% 
    group_split(direction) %>% 
    purrr::map_dfr(., add_xpnsl_ranks) %>% 
    mutate(xpnsl_rank = ifelse(direction == 'None', NA, xpnsl_rank))

#### Correlation between observed and permuted XP-nSL across iterations

In [ ]:
perm_xpnsl_df <- purrr::map(snakemake@input[["win_xpnsl_perm"]], function(x) suppressMessages(read_delim(x, delim ='\t'))) %>% 
    map_dfr(., assign_xpnsl_outliers)
head(perm_xpnsl_df)

In [ ]:
calc_cor <- function(df){
    val <- cor(df$mean, obs_xpnsl_df_filt$mean, method = "pearson")
    return(data.frame(cor = val))
}
cor_plot <- perm_xpnsl_df %>% 
    group_split(iter) %>% 
    purrr::map_dfr(., calc_cor) %>% 
    ggplot(aes(x = cor)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of iterations") + xlab("Permuted vs. observed XP-nSL Pearson's correlation coefficient") +
        theme_classic() +
        theme(axis.title = element_text(size = 17),
              axis.text = element_text(size = 15))

options(repr.plot.width = 8, repr.plot.height = 8)
cor_plot
ggsave(filename = snakemake@output[["cor_plot"]], plot = cor_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

#### Permutations and observed density plots

In [ ]:
urban_sel_permuted <- perm_xpnsl_df %>% filter(direction == "Urban")
urban_sel_observed <- obs_xpnsl_df_filt %>% filter(direction == "Urban")

urb_mean_plot <- ggplot() +
    geom_density(data = urban_sel_permuted, aes(x = mean, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = urban_sel_observed, aes(x = mean, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Mean windowed XP-nSL score") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    theme_classic() +
    theme(axis.title = element_text(size = 17),
          axis.text = element_text(size = 15))

urb_mean_plot
ggsave(filename = snakemake@output[["urb_mean_plot"]], plot = urb_mean_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
urb_prop_plot <- ggplot() +
    geom_density(data = urban_sel_permuted, aes(x = prop_outlier, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = urban_sel_observed, aes(x = prop_outlier, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Proportion of XP-nSL scores in window > 2") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#0070ec", "Observed" = "#003876")) +
    theme_classic() +
    theme(axis.title = element_text(size = 17),
          axis.text = element_text(size = 15))

urb_prop_plot
ggsave(filename = snakemake@output[["urb_prop_plot"]], plot = urb_prop_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
# Dataframe with percentile in permuted distribution of each observed XP-nSL window
urban_percentile <- urban_sel_observed %>% 
    rowwise() %>% 
    mutate(mean_perm_perc = sum(mean >= urban_sel_permuted$mean) / nrow(urban_sel_permuted),
           prop_perm_perc = sum(prop_outlier >= urban_sel_permuted$prop_outlier) / nrow(urban_sel_permuted)) %>% 
    arrange(desc(mean_perm_perc)) %>% 
    dplyr::select(Chr, winID, start, end, winCenter, mean_perm_perc, prop_perm_perc)

In [ ]:
rural_sel_permuted <- perm_xpnsl_df %>% filter(direction == "Rural")
rural_sel_observed <- obs_xpnsl_df_filt %>% filter(direction == "Rural")

rur_mean_plot <- ggplot() +
    geom_density(data = rural_sel_permuted, aes(x = mean, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = rural_sel_observed, aes(x = mean, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Mean windowed XP-nSL score") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    theme_classic() +
    theme(axis.title = element_text(size = 17),
          axis.text = element_text(size = 15))

rur_mean_plot
ggsave(filename = snakemake@output[["rur_mean_plot"]], plot = rur_mean_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
rur_prop_plot <- ggplot() +
    geom_density(data = rural_sel_permuted, aes(x = prop_outlier, fill = "Permuted", color = "Permuted"), alpha = 0.5) + 
    geom_density(data = rural_sel_observed, aes(x = prop_outlier, fill = "Observed", color = "Observed"), alpha = 0.5) +
    ylab("Density") + xlab("Proportion of XP-nSL scores < -2") +
    scale_fill_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    scale_color_manual(name = "Distribution", values = c("Permuted" = "#00e989", "Observed" = "#007243")) +
    theme_classic() +
    theme(axis.title = element_text(size = 17),
          axis.text = element_text(size = 15))

rur_prop_plot
ggsave(filename = snakemake@output[["rur_prop_plot"]], plot = rur_prop_plot, width = 8, 
       height = 8, units = "in", device = "pdf", dpi = 600)

In [ ]:
# Dataframe with percentile in permuted distribution of each observed XP-nSL window
rural_percentile <- rural_sel_observed %>% 
    rowwise() %>% 
    mutate(mean_perm_perc = sum(mean <= rural_sel_permuted$mean) / nrow(rural_sel_permuted),
           prop_perm_perc = sum(prop_outlier >= rural_sel_permuted$prop_outlier) / nrow(rural_sel_permuted)) %>% 
    arrange(desc(mean_perm_perc)) %>% 
    dplyr::select(Chr, winID, start, end, winCenter, mean_perm_perc, prop_perm_perc)

In [ ]:
obs_xpnsl_df_filt <- obs_xpnsl_df_filt %>% 
    left_join(., bind_rows(urban_percentile, rural_percentile), by = c("Chr", "winID", "start", "end", "winCenter"))
head(obs_xpnsl_df_filt)

write_delim(obs_xpnsl_df_filt, snakemake@output[["xpnsl_df"]], delim='\t')

### Merge XP-nSL outlier windows and get top 10

In [ ]:
# Function to merge consecutive outlier windows
merge_xpnsl_windows <- function(df, win_dist = 0){

    dir <- df %>% pull(direction) %>% unique()
    df_sorted <- df %>% 
        dplyr::select(Chr, start, end, everything()) %>% 
        arrange(Chr, start) %>% 
        mutate(prop_outlier = round(prop_outlier, 3),
               mean_perm_perc = round(mean_perm_perc, 3),
               prop_perm_perc = round(prop_outlier, 3))

    col_names <- c('Chr', 'start', 'end', 'mean_xpnsl', 'min_max', 'direction', 'mean_prop_outlier', 'all_prop_outlier', 'mean_percentile_permuted', 'prop_percentile_permuted','min_xpnsl_rank', 'all_xpnsl_ranks')
    if(dir == 'Urban'){
        cols <- c('6,7,17,18,18,20,21,19,19')
        operation <- c('mean,max,distinct,mean,collapse,collapse,collapse,min,collapse')
    }else if(dir == 'Rural'){
        cols <- c('6,8,17,18,18,20,21,19,19')
        operation <- c('mean,min,distinct,mean,collapse,collapse,collapse,min,collapse')
    }
    
    df_merged  <- bt.merge(i = df_sorted, c = cols, o = operation, d = win_dist)
    names(df_merged) <- col_names
    df_merged <- df_merged %>% 
        mutate(win_size = end - start)

    print(sprintf('%s: There were %s XP-nSL outlier windows prior to merging. There are %s outlier regions after merging consecutive outlier windows', dir, nrow(df_sorted), nrow(df_merged)))
    return(df_merged)
}

win_xpnsl_df_filt_merged <- obs_xpnsl_df_filt %>% 
    filter(direction != 'None') %>% 
    group_split(direction) %>% 
    purrr::map_dfr(., merge_xpnsl_windows)

In [ ]:
xpnsl_top10_urban_rural <- win_xpnsl_df_filt_merged %>% 
    filter(min_xpnsl_rank <= 10) %>% 
    group_by(direction) %>% 
    arrange(min_xpnsl_rank, .by_group = TRUE) %>% 
    dplyr::select(Chr, start, end, win_size, direction, mean_xpnsl, min_max, all_prop_outlier, mean_prop_outlier, mean_percentile_permuted, prop_percentile_permuted, all_xpnsl_ranks, min_xpnsl_rank) %>% 
    ungroup()
xpnsl_top10_urban_rural

## nSL

- I estimated nSL in the same 50 Kb windows as above, separately in urban and rural habitats
- I'll do the same thing as I did above, minus the permutations

In [ ]:
nsl_df <- read_delim(snakemake@input[["win_nsl"]], delim='\t')
head(nsl_df)

In [ ]:
nsl_nsites_thresh <- 25
nsl_nsites_hist <- nsl_df %>% 
    ggplot(aes(x = n)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of windows") + xlab("Number of sites") +
        geom_vline(xintercept = nsl_nsites_thresh, color = "red", linetype = "dashed", linewidth = 1) +
        theme_classic() +
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 17))

nsl_nsites_hist
ggsave(filename = snakemake@output[["nsl_nSites_hist"]], plot = nsl_nsites_hist, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
# Function to assign outliers to nSL windows
assign_nsl_outliers <- function(df, thresh = nsl_nsites_thresh){

    df_filt <- df %>%
        filter(n >= thresh)

    # Get critical values for mean XP-nSL score and proportions greater or lesser than 2 and -2, respectively
    score_quant <- quantile(df_filt %>% pull(mean), probs = 0.99)
    gtprop_quant <- quantile(df_filt %>% pull(gt_frac), probs = 0.99)

    # Identify outliers and add as categorical variable to windows dataframe
    df_filt <- df_filt %>%
        mutate(score_outlier = ifelse(mean >= score_quant, 1, 0),
               gtprop_outlier = ifelse(gt_frac >= gtprop_quant, 1, 0),
               is_outlier = ifelse(score_outlier == 1 & gtprop_outlier == 1, 1, 0))

    return(df_filt)
}

nsl_df_filt <- nsl_df %>% 
    group_split(habitat) %>% 
    purrr::map_dfr(., assign_nsl_outliers, thresh = nsl_nsites_thresh)

sprintf("%s of %s nSL windows remaining after removing those with less than %s sites", nrow(nsl_df_filt), nrow(nsl_df), nsl_nsites_thresh)

head(nsl_df_filt)

In [ ]:
plot_single_population_hapstat_win_manhat <- function(df, hab, stat){

    df <- df %>% filter(habitat == hab)
    
    # Setting up cummulative genome-wide x-axis
    data_cum <- df %>% 
        group_by(Chr) %>% 
        summarise(max_winCenter = max(winCenter)) %>% 
        mutate(winCenter_add = lag(cumsum(max_winCenter), default = 0)) %>% 
        dplyr::select(Chr, winCenter_add)
    
    df_mod <- df %>%
        inner_join(data_cum, by = "Chr") %>% 
        mutate(winCenter_cum = winCenter + winCenter_add) 
    
    axis_set <- df_mod %>% 
      group_by(Chr) %>% 
      summarize(center = mean(winCenter_cum))

    # Get Fst outliers and genome-wide critical value
    outliers <- df_mod %>% filter(is_outlier == 1) 
    quant <- quantile(abs(df %>% pull(mean)), probs = c(0.99))

    if(hab == 'Urban'){
        col = '#003876'
    } else {
        col = '#007243'
    }

    if(stat == 'nSL'){
        ylab = '| nSL |'
        coord = c(0, 4)
        breaks = seq(0, 4, 1)
    } else if (stat == 'iHS') {
        ylab = '| iHS |'
        coord = c(0, 3.25)
        breaks = seq(0, 3, 1)
    } else {
        ylab = 'iHH12'
        coord = c(-0.5, 17)
        breaks = seq(0, 16, 2)
    }

    # Generate Manhattan plot
    manhat_plot <- df_mod %>%
        filter(is_outlier != 1) %>% 
        mutate(chrom_cat = case_when(Chr == 'Chr01_Occ' ~ 'One',
                                     Chr == 'Chr01_Pall' ~ 'Two',
                                     Chr == 'Chr02_Occ' ~ 'One',
                                     Chr == 'Chr02_Pall' ~ 'Two',
                                     Chr == 'Chr03_Occ' ~ 'One',
                                     Chr == 'Chr03_Pall' ~ 'Two',
                                     Chr == 'Chr04_Occ' ~ 'One',
                                     Chr == 'Chr04_Pall' ~ 'Two',
                                     Chr == 'Chr05_Occ' ~ 'One',
                                     Chr == 'Chr05_Pall' ~ 'Two',
                                     Chr == 'Chr06_Occ' ~ 'One',
                                     Chr == 'Chr06_Pall' ~ 'Two',
                                     Chr == 'Chr07_Occ' ~ 'One',
                                     Chr == 'Chr07_Pall' ~ 'Two',
                                     Chr == 'Chr08_Occ' ~ 'One',
                                     Chr == 'Chr08_Pall' ~ 'Two')) %>%
            ggplot(aes(x = winCenter_cum, y = mean)) +
            geom_point(shape = 21, alpha = 0.4, size = 3, aes(fill = chrom_cat, color = chrom_cat)) +
            geom_point(data = outliers, shape = 21, alpha = 1, size = 3, color = col, fill = col) +
            geom_hline(yintercept = quant, color = "grey40", linetype = "dashed") +
            scale_x_continuous(label = axis_set$Chr, breaks=axis_set$center) +
            scale_y_continuous(expand = c(0,0), breaks = breaks) +
            coord_cartesian(ylim = coord) +
            scale_fill_manual(values = c("black", "grey40")) + 
            scale_color_manual(values = c("black", "grey40")) + 
            ylab(ylab) + xlab('') +
            ggtitle(sprintf("%s Windowed %s", hab, stat)) +
            theme_classic() +
            theme(
                legend.position = "none",
                panel.border = element_blank(),
                panel.grid.major.x = element_blank(),
                panel.grid.minor.x = element_blank(),
                axis.text = element_text(size=16),
                axis.title = element_text(size=20),
                axis.text.x = element_text(angle = 45, hjust = 1),
                plot.title = element_text(size = 20, face = "bold")
              )
    return(manhat_plot)
}

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
rural_nsl_manhat <- plot_single_population_hapstat_win_manhat(nsl_df_filt, "Rural", stat = 'nSL')
rural_nsl_manhat
ggsave(filename = snakemake@output[["rur_nsl_manhat"]], plot = rural_nsl_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
nsl_persite <- snakemake@input[["norm_nsl"]] %>% 
    purrr::map_dfr(., load_hapstat_norm, stat = "nsl") %>% 
    mutate(crit = ifelse(normnsl < -2 | normnsl > 2, 1, 0))

In [ ]:
plot_single_population_hapstat_persite_manhat <- function(df, hab, stat){

    df <- df %>% filter(habitat == hab)
    
    # Setting up cummulative genome-wide x-axis
    data_cum <- df %>% 
        group_by(Chr) %>% 
        summarise(max_pos = max(pos)) %>% 
        mutate(pos_add = lag(cumsum(max_pos), default = 0)) %>% 
        dplyr::select(Chr, pos_add)
    
    df_mod <- df %>%
        inner_join(data_cum, by = "Chr") %>% 
        mutate(pos_cum = pos + pos_add) 
    
    axis_set <- df_mod %>% 
      group_by(Chr) %>% 
      summarize(center = mean(pos_cum))

    # Get Fst outliers and genome-wide critical value
    outliers <- df_mod %>% filter(crit == 1) 

    if(hab == 'Urban'){
        col = '#003876'
    } else {
        col = '#007243'
    }

    if(stat == 'nSL'){
        y = "normnsl"
        coord = c(-6, 6)
        breaks = seq(-6, 6, 2)
    } else if (stat == 'iHS') {
        y = "normihs"
        coord = c(-8, 8)
        breaks = seq(-8, 8, 2)
    } else {
        y = "normihh12"
        coord = c(-0.5, 120)
        breaks = seq(0, 120, 20)
    }

    # Generate Manhattan plot
    manhat_plot <- df_mod %>%
        filter(crit == 0) %>% 
        mutate(chrom_cat = case_when(Chr == 'Chr01_Occ' ~ 'One',
                                     Chr == 'Chr01_Pall' ~ 'Two',
                                     Chr == 'Chr02_Occ' ~ 'One',
                                     Chr == 'Chr02_Pall' ~ 'Two',
                                     Chr == 'Chr03_Occ' ~ 'One',
                                     Chr == 'Chr03_Pall' ~ 'Two',
                                     Chr == 'Chr04_Occ' ~ 'One',
                                     Chr == 'Chr04_Pall' ~ 'Two',
                                     Chr == 'Chr05_Occ' ~ 'One',
                                     Chr == 'Chr05_Pall' ~ 'Two',
                                     Chr == 'Chr06_Occ' ~ 'One',
                                     Chr == 'Chr06_Pall' ~ 'Two',
                                     Chr == 'Chr07_Occ' ~ 'One',
                                     Chr == 'Chr07_Pall' ~ 'Two',
                                     Chr == 'Chr08_Occ' ~ 'One',
                                     Chr == 'Chr08_Pall' ~ 'Two')) %>%
            ggplot(aes(x = pos_cum, y = !!sym(y))) +
            geom_point(shape = 21, alpha = 0.1, size = 0.25, aes(fill = chrom_cat, color = chrom_cat)) +
            geom_point(data = outliers, shape = 21, alpha = 0.1, size = 0.25, color = col, fill = col) +
            geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
            geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
            scale_x_continuous(label = axis_set$Chr, breaks=axis_set$center) +
            scale_y_continuous(expand = c(0,0), breaks = breaks) +
            coord_cartesian(ylim = coord) +
            scale_fill_manual(values = c("black", "grey40")) + 
            scale_color_manual(values = c("black", "grey40")) + 
            ylab(sprintf('Normalized %s', stat)) + xlab('') +
            ggtitle(sprintf("%s Per-site %s", hab, stat)) +
            theme_classic() +
            theme(
                legend.position = "none",
                panel.border = element_blank(),
                panel.grid.major.x = element_blank(),
                panel.grid.minor.x = element_blank(),
                axis.text = element_text(size=16),
                axis.title = element_text(size=20),
                axis.text.x = element_text(angle = 45, hjust = 1),
                plot.title = element_text(size = 20, face = "bold")
              )
    return(manhat_plot)
}

In [ ]:
rural_nsl_persite_manhat <- plot_single_population_hapstat_persite_manhat(nsl_persite, "Rural", stat = "nSL")
rural_nsl_persite_manhat
ggsave(filename = snakemake@output[["rur_nsl_manhat_persite"]], plot = rural_nsl_persite_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["rur_nsl_manhat_persite_png"]], plot = rural_nsl_persite_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

In [ ]:
urban_nsl_manhat <- plot_single_population_hapstat_win_manhat(nsl_df_filt, "Urban", stat = 'nSL')
urban_nsl_manhat
ggsave(filename = snakemake@output[["urb_nsl_manhat"]], plot = urban_nsl_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
urban_nsl_persite_manhat <- plot_single_population_hapstat_persite_manhat(nsl_persite, "Urban", stat = "nSL")
urban_nsl_persite_manhat
ggsave(filename = snakemake@output[["urb_nsl_manhat_persite"]], plot = urban_nsl_persite_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["urb_nsl_manhat_persite_png"]], plot = urban_nsl_persite_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

In [ ]:
nsl_df_filt %>% 
    filter(is_outlier == 1) %>% 
    write_delim(snakemake@output[["nsl_df"]], delim="\t")

## iHS

In [ ]:
ihs_df <- read_delim(snakemake@input[["win_ihs"]], delim='\t')
head(ihs_df)

In [ ]:
ihs_nsites_hist <- ihs_df %>% 
    ggplot(aes(x = n)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of windows") + xlab("Number of sites") +
        geom_vline(xintercept = nsl_nsites_thresh, color = "red", linetype = "dashed", linewidth = 1) +
        theme_classic() +
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 17))

options(repr.plot.width = 8, repr.plot.height = 8)
ihs_nsites_hist
ggsave(filename = snakemake@output[["ihs_nSites_hist"]], plot = ihs_nsites_hist, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
ihs_df_filt <- ihs_df %>% 
    group_split(habitat) %>% 
    purrr::map_dfr(., assign_nsl_outliers, thresh = nsl_nsites_thresh)

sprintf("%s of %s iHS windows remaining after removing those with less than %s sites", nrow(ihs_df_filt), nrow(ihs_df), nsl_nsites_thresh)

head(ihs_df_filt)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
rural_ihs_manhat <- plot_single_population_hapstat_win_manhat(ihs_df_filt, "Rural", stat = 'iHS')
rural_ihs_manhat
ggsave(filename = snakemake@output[["rur_ihs_manhat"]], plot = rural_ihs_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
urban_ihs_manhat <- plot_single_population_hapstat_win_manhat(ihs_df_filt, "Urban", stat = 'iHS')
urban_ihs_manhat
ggsave(filename = snakemake@output[["urb_ihs_manhat"]], plot = urban_ihs_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
ihs_persite <- snakemake@input[["norm_ihs"]] %>% 
    purrr::map_dfr(., load_hapstat_norm, stat = "ihs") %>% 
    mutate(crit = ifelse(normihs < -2 | normihs > 2, 1, 0))

In [ ]:
urban_ihs_persite_manhat <- plot_single_population_hapstat_persite_manhat(ihs_persite, "Urban", stat = "iHS")
urban_ihs_persite_manhat
ggsave(filename = snakemake@output[["urb_ihs_manhat_persite"]], plot = urban_ihs_persite_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["urb_ihs_manhat_persite_png"]], plot = urban_ihs_persite_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

In [ ]:
rural_ihs_persite_manhat <- plot_single_population_hapstat_persite_manhat(ihs_persite, "Rural", stat = "iHS")
rural_ihs_persite_manhat
ggsave(filename = snakemake@output[["rur_ihs_manhat_persite"]], plot = rural_ihs_persite_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["rur_ihs_manhat_persite_png"]], plot = rural_ihs_persite_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

In [ ]:
ihs_df_filt %>% 
    filter(is_outlier == 1) %>% 
    write_delim(snakemake@output[["ihs_df"]], delim="\t")

## iHH12

In [ ]:
ihh12_df <- read_delim(snakemake@input[["win_ihh12"]], delim='\t')
head(ihh12_df)

In [ ]:
ihh12_nsites_hist <- ihh12_df %>% 
    ggplot(aes(x = n)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of windows") + xlab("Number of sites") +
        geom_vline(xintercept = nsl_nsites_thresh, color = "red", linetype = "dashed", linewidth = 1) +
        theme_classic() +
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 17))

options(repr.plot.width = 8, repr.plot.height = 8)
ihh12_nsites_hist
ggsave(filename = snakemake@output[["ihh12_nSites_hist"]], plot = ihh12_nsites_hist, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
ihh12_df_filt <- ihh12_df %>% 
    group_split(habitat) %>% 
    purrr::map_dfr(., assign_nsl_outliers, thresh = nsl_nsites_thresh)

sprintf("%s of %s iHH12 windows remaining after removing those with less than %s sites", nrow(ihh12_df_filt), nrow(ihh12_df), nsl_nsites_thresh)

head(ihh12_df_filt)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6)
urban_ihh12_manhat <- plot_single_population_hapstat_win_manhat(ihh12_df_filt, "Urban", stat = 'iHH12')
urban_ihh12_manhat
ggsave(filename = snakemake@output[["urb_ihh12_manhat"]], plot = urban_ihh12_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
rural_ihh12_manhat <- plot_single_population_hapstat_win_manhat(ihh12_df_filt, "Rural", stat = 'iHH12')
rural_ihh12_manhat
ggsave(filename = snakemake@output[["rur_ihh12_manhat"]], plot = rural_ihh12_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
ihh12_persite <- snakemake@input[["norm_ihh12"]] %>% 
    purrr::map_dfr(., load_hapstat_norm, stat = "ihh12") %>% 
    mutate(crit = ifelse(normihh12 < -2 | normihh12 > 2, 1, 0))

In [ ]:
urban_ihh12_persite_manhat <- plot_single_population_hapstat_persite_manhat(ihh12_persite, "Urban", stat = "iHH12")
urban_ihh12_persite_manhat
ggsave(filename = snakemake@output[["urb_ihh12_manhat_persite"]], plot = urban_ihh12_persite_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["urb_ihh12_manhat_persite_png"]], plot = urban_ihh12_persite_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

In [ ]:
rural_ihh12_persite_manhat <- plot_single_population_hapstat_persite_manhat(ihh12_persite, "Rural", stat = "iHH12")
rural_ihh12_persite_manhat
ggsave(filename = snakemake@output[["rur_ihh12_manhat_persite"]], plot = rural_ihh12_persite_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")
ggsave(filename = snakemake@output[["rur_ihh12_manhat_persite_png"]], plot = rural_ihh12_persite_manhat, 
       height = 8, width = 20, device = "png", dpi = 100, units = "in")

In [ ]:
ihh12_df_filt %>% 
    filter(is_outlier == 1) %>% 
    write_delim(snakemake@output[["ihh12_df"]], delim="\t")

## Fst

In [ ]:
gt_fst_df <- read_delim(snakemake@input[["gt_win_fst"]]) %>% 
    filter(pop1 == "Urban" & pop2 == "Rural") %>%
    mutate(winCenter = (window_pos_1 - 1) + 25000) %>% 
    rename("Chr" = "chromosome",
           "start" = "window_pos_1",
           "end" = "window_pos_2")
head(gt_fst_df)

In [ ]:
gt_fst_nsites_hist <- gt_fst_df %>% 
    ggplot(aes(x = no_snps)) +
        geom_histogram(bins = 50, color = "black", fill = "white") +
        ylab("Number of windows") + xlab("Number of sites") +
        geom_vline(xintercept = xpnsl_nsites_thresh, color = "red", linetype = "dashed", linewidth = 1) +
        theme_classic() +
        theme(axis.text = element_text(size = 15),
              axis.title = element_text(size = 17))

options(repr.plot.width = 8, repr.plot.height = 8)
gt_fst_nsites_hist
ggsave(filename = snakemake@output[["gt_fst_nSites_hist"]], plot = gt_fst_nsites_hist, 
       height = 8, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
gt_fst_df_filt <- gt_fst_df %>% 
    filter(no_snps >= xpnsl_nsites_thresh) %>% 
    mutate(avg_hudson_fst = ifelse(avg_hudson_fst < 0, 0, avg_hudson_fst))

gt_fst_quant <- quantile(gt_fst_df_filt %>% pull(avg_hudson_fst), probs = 0.99)

gt_fst_df_filt <- gt_fst_df_filt %>% 
    mutate(is_outlier = ifelse(avg_hudson_fst >= gt_fst_quant, 1, 0))

In [ ]:
range(gt_fst_df_filt$avg_hudson_fst)

In [ ]:
# Setting up cummulative genome-wide x-axis
data_cum <- gt_fst_df_filt %>% 
    group_by(Chr) %>% 
    summarise(max_winCenter = max(winCenter)) %>% 
    mutate(winCenter_add = lag(cumsum(max_winCenter), default = 0)) %>% 
    dplyr::select(Chr, winCenter_add)

gt_fst_df_filt_mod <- gt_fst_df_filt %>%
    inner_join(data_cum, by = "Chr") %>% 
    mutate(winCenter_cum = winCenter + winCenter_add) 

axis_set <- gt_fst_df_filt_mod %>% 
  group_by(Chr) %>% 
  summarize(center = mean(winCenter_cum))

# Get XP-nSL outliers for plotting as separate layers
gt_fst_not_outlier <- gt_fst_df_filt_mod %>% filter(is_outlier == 0)
gt_fst_outliers <- gt_fst_df_filt_mod %>% filter(is_outlier == 1)

gt_fst_not_outlier <- gt_fst_not_outlier %>%
    mutate(chrom_cat = case_when(Chr == 'Chr01_Occ' ~ 'One',
                                 Chr == 'Chr01_Pall' ~ 'Two',
                                 Chr == 'Chr02_Occ' ~ 'One',
                                 Chr == 'Chr02_Pall' ~ 'Two',
                                 Chr == 'Chr03_Occ' ~ 'One',
                                 Chr == 'Chr03_Pall' ~ 'Two',
                                 Chr == 'Chr04_Occ' ~ 'One',
                                 Chr == 'Chr04_Pall' ~ 'Two',
                                 Chr == 'Chr05_Occ' ~ 'One',
                                 Chr == 'Chr05_Pall' ~ 'Two',
                                 Chr == 'Chr06_Occ' ~ 'One',
                                 Chr == 'Chr06_Pall' ~ 'Two',
                                 Chr == 'Chr07_Occ' ~ 'One',
                                 Chr == 'Chr07_Pall' ~ 'Two',
                                 Chr == 'Chr08_Occ' ~ 'One',
                                 Chr == 'Chr08_Pall' ~ 'Two'))

gt_fst_manhat <- ggplot() +
        geom_point(data = gt_fst_not_outlier, shape = 21, alpha = 0.4, size = 3, 
                   aes(x = winCenter_cum, y = avg_hudson_fst, fill = chrom_cat, color = chrom_cat)) +
        geom_point(data = gt_fst_outliers, shape = 21, alpha = 1, size = 3, color = 'red', fill = 'red',
                   aes(x = winCenter_cum, y = avg_hudson_fst)) +
        geom_hline(yintercept = gt_fst_quant, color = "grey40", linetype = "dashed") +
        scale_x_continuous(label = axis_set$Chr, breaks = axis_set$center) +
        scale_y_continuous(expand = c(0,0), breaks = seq(0, 0.06, 0.02)) +
        coord_cartesian(ylim = c(0, 0.06)) +
        scale_fill_manual(values = c("black", "grey40")) + 
        scale_color_manual(values = c("black", "grey40")) + 
        ylab('Fst') + xlab('Chromosomes') +
        ggtitle('Urban-Rural Fst') +
        theme_classic() +
        theme(
            legend.position = "none",
            panel.border = element_blank(),
            panel.grid.major.x = element_blank(),
            panel.grid.minor.x = element_blank(),
            axis.text = element_text(size=16),
            axis.title = element_text(size=20),
            axis.text.x = element_text(angle = 45, hjust = 1),
            plot.title = element_text(size=20, face="bold")
          )

options(repr.plot.width = 20, repr.plot.height = 6)
gt_fst_manhat
ggsave(filename = snakemake@output[["gt_fst_manhat"]], plot = gt_fst_manhat, 
       height = 8, width = 20, device = "pdf", dpi = 600, units = "in")

In [ ]:
gt_fst_df_filt %>% 
    filter(is_outlier == 1) %>% 
    write_delim(snakemake@output[["gt_fst_df"]], delim="\t")

## Overlap between statistics

### Venn diagram

In [ ]:
xpnsl_urban_outliers <- obs_xpnsl_df_filt %>% filter(direction == "Urban") %>% mutate(id = paste0(Chr, '__', winCenter)) 
nsl_urban_outlier <- nsl_df_filt %>% filter(habitat == "Urban" & is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))
ihs_urban_outlier <- ihs_df_filt %>% filter(habitat == "Urban" & is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))
ihh12_urban_outlier <- ihh12_df_filt %>% filter(habitat == "Urban" & is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))
gt_fst_outlier <- gt_fst_df_filt %>% filter(is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))

In [ ]:
urban_outliers_list <- list(xpnsl=xpnsl_urban_outliers %>% pull(id),
                            nsl=nsl_urban_outlier %>% pull(id),
                            ihs=ihs_urban_outlier %>% pull(id),
                            ihh12=ihh12_urban_outlier %>% pull(id),
                            fst=gt_fst_outlier %>% pull(id))

In [ ]:
ggVennDiagram(urban_outliers_list, label = "count")

In [ ]:
xpnsl_rural_outliers <- obs_xpnsl_df_filt %>% filter(direction == "Rural") %>% mutate(id = paste0(Chr, '__', winCenter))
nsl_rural_outlier <- nsl_df_filt %>% filter(habitat == "Rural" & is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))
ihs_rural_outlier <- ihs_df_filt %>% filter(habitat == "Rural" & is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))
ihh12_rural_outlier <- ihh12_df_filt %>% filter(habitat == "Rural" & is_outlier == 1) %>% mutate(id = paste0(Chr, '__', winCenter))

In [ ]:
rural_outliers_list <- list(xpnsl=xpnsl_rural_outliers %>% pull(id),
                            nsl=nsl_rural_outlier %>% pull(id),
                            ihs=ihs_rural_outlier %>% pull(id),
                            ihh12=ihh12_rural_outlier %>% pull(id),
                            fst=gt_fst_outlier %>% pull(id))

In [ ]:
ggVennDiagram(rural_outliers_list, label = "count")

In [ ]:
xpnsl_top10_urban_rural

In [ ]:
get_positions <- function(df){
    df_out <- df %>% 
        dplyr::select(Chr, start, end)
    return(df_out)
}

In [ ]:
xpnsl_top10_positions <- get_positions(xpnsl_top10_urban_rural)
nsl_urban_positions <- get_positions(nsl_urban_outlier)
nsl_rural_positions <- get_positions(nsl_rural_outlier)
ihs_urban_positions <- get_positions(ihs_urban_outlier)
ihs_rural_positions <- get_positions(ihs_rural_outlier)

In [ ]:
count_overlapping_outlier_windows <- function(pos1, pos2){
    
    # Count number of overlapping statistic outlier windows in each XP-nSL window
    num_outliers <- bt.intersect(pos1, pos2, c = T) %>% pull(V4)
    return(num_outliers)
} 

In [ ]:
xpnsl_top10_urban_rural_with_other_stats <- xpnsl_top10_urban_rural %>% 
    mutate(num_nsl_urban = count_overlapping_outlier_windows(get_positions(.), get_positions(nsl_urban_outlier)),
           num_nsl_rural = count_overlapping_outlier_windows(get_positions(.), get_positions(nsl_rural_outlier)),
           num_ihs_urban = count_overlapping_outlier_windows(get_positions(.), get_positions(ihs_urban_outlier)),
           num_ihs_rural = count_overlapping_outlier_windows(get_positions(.), get_positions(ihs_rural_outlier)),
           num_ihh12_urban = count_overlapping_outlier_windows(get_positions(.), get_positions(ihh12_urban_outlier)),
           num_ihh12_rural = count_overlapping_outlier_windows(get_positions(.), get_positions(ihh12_rural_outlier)),
           num_fst = count_overlapping_outlier_windows(get_positions(.), get_positions(gt_fst_outlier)))

# Create output tables

## Dataframe with gene ID in top 10 selected regions

In [ ]:
# Load GFF
gff <- ape::read.gff(snakemake@input[['gff']], GFF3 = TRUE) %>% 
    dplyr::select(seqid, start, end, everything())

# Get gene names dataframe
gene_names <- gff %>% 
    filter(type == 'gene') %>% 
    mutate(gene = str_extract(attributes, pattern = '(?<=gene=)\\w+(?=;)'),
           gene_id = str_extract(attributes, pattern = '(?<=ID\\=)ACLI19_g\\d+(?=;)')) %>% 
    dplyr::select(seqid, start, end, gene_id, gene)

# Get gene products and GO annotations dataframe
prod_go_annot <- gff %>% 
    filter(type == 'mRNA') %>% 
    mutate(id = str_extract(attributes, pattern = '(?<=ID\\=)ACLI19_g\\d+\\.t\\d+(?=;)'),
           func = str_extract(attributes, pattern = '(?<=product=)[^;]*'),
           go = str_extract_all(attributes, pattern = 'GO:\\d+(?=(,|;))')) %>% 
    separate(id, into = c('gene_id', 'trans'), sep = '\\.') %>% 
    filter(trans == 't1') %>% 
    dplyr::select(seqid, gene_id, func, go)

# Combine genes, functions, and GO annotations into single dataframe
genes_prods_go_df <- left_join(gene_names, prod_go_annot, by = c('seqid', 'gene_id')) %>% 
    rename('Chr' = 'seqid')

In [ ]:
xpnsl_top10_urban_rural_with_genes_long <- bt.intersect(xpnsl_top10_urban_rural %>% 
                                                            dplyr::select(Chr, start, end, direction), 
                                                        genes_prods_go_df %>% 
                                                            dplyr::select(-go),
                                                        wa=T, wb=T) %>% 
    dplyr::select(V1, V2, V3, V4, V8, V9, V10)
names(xpnsl_top10_urban_rural_with_genes_long) <- c('Chr', 'start', 'end', 'direction', 'gene_id', 'gene_symbol', 'product')
xpnsl_top10_urban_rural_with_genes_long

In [ ]:
# Write dataframe with gene IDs for top ten selected urban and rural regions
write_delim(xpnsl_top10_urban_rural_with_genes_long, snakemake@output[['top_ten_genes']], delim = '\t')

## Table with gene symbols, products, and Fst for top selected regions

In [ ]:
# Concatenate gene symbols for top 10 selected regions
symbols <- xpnsl_top10_urban_rural_with_genes_long %>% 
    dplyr::select(-gene_id, -product) %>% 
    filter(!is.na(gene_symbol)) %>% 
    group_by(Chr, start, end, direction) %>% 
    summarise(gene_symbols = toString(gene_symbol))

# Concatenate products for top 10 selected regions
prods <- xpnsl_top10_urban_rural_with_genes_long %>% 
    dplyr::select(-gene_id, -gene_symbol) %>% 
    filter(product != 'hypothetical protein') %>% 
    group_by(Chr, start, end, direction) %>% 
    summarise(products = toString(product))

# Add symbols and products to table
xpnsl_top10_urban_rural_with_genes <- xpnsl_top10_urban_rural_with_other_stats %>% 
    left_join(., symbols) %>% 
    left_join(., prods)

xpnsl_top10_urban_rural_with_genes

In [ ]:
# Write table with gene symbols, products, and Fst overlaps for top ten selected yrban and rural regions
write_delim(xpnsl_top10_urban_rural_with_genes, snakemake@output[['top_ten_tbl']], delim = '\t')

## Zoomed-in Manhattan plots

### Positive selection in urban habitats

In [ ]:
xpnsl_top10_urban_rural_with_other_stats %>% 
    filter(direction == 'Urban') %>% 
    dplyr::select(Chr, start, end, win_size, min_xpnsl_rank, num_nsl_urban:num_fst)

In [ ]:
max_xpnsl <- xpnsl_persite %>% filter(normxpnsl == max(normxpnsl))
max_xpnsl

In [ ]:
max <- max_xpnsl %>% dplyr::select(Chr, pos) %>% rename('start' = 'pos') %>% mutate(end = start) %>% dplyr::select(Chr, start, end)
genes <- genes_prods_go_df %>% dplyr::select(Chr, start, end, gene, func)
bt.intersect(max, genes, wb=T)

#### Chr04_Occ

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Urban') %>% 
    filter(Chr == 'Chr04_Occ') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_urban_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- xpnsl_persite %>%
    filter(Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

xpnsl_not_outlier <- region_raw_df %>% filter(crit == 0)
urban_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl < 0)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}

fst_outliers <- gt_fst_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
fst_wins <- bt.intersect(fst_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
nsl_outliers <- nsl_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
nsl_wins <- bt.intersect(nsl_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
ihh12_outliers <- ihh12_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
ihh12_wins <- bt.intersect(ihh12_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')

manhat_plot <- ggplot(xpnsl_not_outlier, aes(x = pos, y = normxpnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#003876', fill = '#003876',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#007243', fill = '#007243',
               aes(x = pos, y = normxpnsl)) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 04_Occ (position in Mbp)') + ylab('Normalized XP-nSL') +
    coord_cartesian(ylim = c(-3.5, 6.5)) +
    geom_segment(data = fst_wins, aes(x = start, xend = end), y = -3, yend = -3, color = '#d62828', linewidth = 1) +
    geom_segment(data = nsl_wins, aes(x = start, xend = end), y = -3.25, yend = -3.25, color = '#ffbe0b', linewidth = 1) +
    geom_segment(data = ihh12_wins, aes(x = start, xend = end), y = -3.75, yend = -3.75, color = '#386641', linewidth = 1) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 500000), labels = formatter1e6) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
ggsave(filename = snakemake@output[["Chr04_Occ_urb_xpnsl"]], plot = manhat_plot, 
       height = 6, width = 8, device = "pdf", dpi = 600, units = "in")

#### Chr05_Occ

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Urban') %>% 
    filter(Chr == 'Chr05_Occ') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_urban_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- xpnsl_persite %>%
    filter(Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

xpnsl_not_outlier <- region_raw_df %>% filter(crit == 0)
urban_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl < 0)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}

fst_outliers <- gt_fst_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
fst_wins <- bt.intersect(fst_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
nsl_outliers <- nsl_df_filt %>% filter(is_outlier == 1 & habitat == "Urban") %>% dplyr::select(Chr, start, end)
nsl_wins <- bt.intersect(nsl_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
ihs_outliers <- ihs_df_filt %>% filter(is_outlier == 1 & habitat == "Urban") %>% dplyr::select(Chr, start, end)
ihs_wins <- bt.intersect(ihs_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
ihh12_outliers <- ihh12_df_filt %>% filter(is_outlier == 1 & habitat == "Urban") %>% dplyr::select(Chr, start, end)
ihh12_wins <- bt.intersect(ihh12_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')

manhat_plot <- ggplot(xpnsl_not_outlier, aes(x = pos, y = normxpnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +   
    geom_point(shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#003876', fill = '#003876',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#007243', fill = '#007243',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = max_xpnsl, shape = 23, size = 2, fill = 'yellow', alpha = 1) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 05_Occ (position in Mbp)') + ylab('Normalized XP-nSL') +
    coord_cartesian(ylim = c(-3.5, 6.5)) +
    geom_segment(data = fst_wins, aes(x = start, xend = end), y = -3, yend = -3, color = '#d62828', linewidth = 1) +
    geom_segment(data = nsl_wins, aes(x = start, xend = end), y = -3.25, yend = -3.25, color = '#ffbe0b', linewidth = 1) +
    geom_segment(data = ihs_wins, aes(x = start, xend = end), y = -3.5, yend = -3.5, color = '#8338ec', linewidth = 1) +
    geom_segment(data = ihh12_wins, aes(x = start, xend = end), y = -3.75, yend = -3.75, color = '#386641', linewidth = 1) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 200000), labels = formatter1e6) +
    # scale_y_continuous(breaks = seq(-2, 6, 2)) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
ggsave(filename = snakemake@output[["Chr05_Occ_urb_xpnsl"]], plot = manhat_plot, 
       height = 6, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Urban') %>% 
    filter(Chr == 'Chr05_Occ') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_urban_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- nsl_persite %>%
    filter(habitat == "Urban" & Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

nsl_not_outlier <- region_raw_df %>% filter(crit == 0)
nsl_outliers <- region_raw_df %>% filter(crit == 1)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}

manhat_plot <- ggplot(nsl_not_outlier, aes(x = pos, y = normnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +    
    geom_point(shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = nsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#ffbe0b', fill = '#ffbe0b',
               aes(x = pos, y = normnsl)) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 05_Occ (position in Mbp)') + ylab('Normalized nSL') +
    coord_cartesian(ylim = c(-5.5, 4.5)) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 250000), labels = formatter1e6) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
# ggsave(filename = snakemake@output[["Chr05_Occ_urb_nsl"]], plot = manhat_plot, 
#        height = 6, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
nsl_persite %>% group_by(habitat) %>% filter(normnsl == max(normnsl) | normnsl == min(normnsl))

In [ ]:
nsl_outliers %>% filter(normnsl == max(normnsl) | normnsl == min(normnsl))

### Positive selection in rural habitats

In [ ]:
xpnsl_top10_urban_rural_with_other_stats %>% 
    filter(direction == 'Rural') %>% 
    dplyr::select(Chr, start, end, win_size, min_xpnsl_rank, num_nsl_urban:num_fst)

In [ ]:
min_xpnsl <- xpnsl_persite %>% filter(normxpnsl == min(normxpnsl))
min_xpnsl

In [ ]:
min <- min_xpnsl %>% dplyr::select(Chr, pos) %>% rename('start' = 'pos') %>% mutate(end = start) %>% dplyr::select(Chr, start, end)
bt.intersect(min, genes, wb=T)

#### Chr08_Pall

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Rural') %>% 
    filter(Chr == 'Chr08_Pall') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_rural_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- xpnsl_persite %>%
    filter(Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

xpnsl_not_outlier <- region_raw_df %>% filter(crit == 0)
urban_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl < 0)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}

fst_outliers <- gt_fst_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
fst_wins <- bt.intersect(fst_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
nsl_outliers <- nsl_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
nsl_wins <- bt.intersect(nsl_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
ihs_outliers <- ihs_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
ihs_wins <- bt.intersect(ihs_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')

manhat_plot <- ggplot(xpnsl_not_outlier, aes(x = pos, y = normxpnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#003876', fill = '#003876',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#007243', fill = '#007243',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = min_xpnsl, shape = 23, size = 2, fill = 'yellow', alpha = 1) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 08_Pall (position in Mbp)') + ylab('Normalized XP-nSL') +
    coord_cartesian(ylim = c(-9.5, 3.5)) +
    geom_segment(data = fst_wins, aes(x = start, xend = end), y = -9, yend = -9, color = '#d62828', linewidth = 1) +
    geom_segment(data = nsl_wins, aes(x = start, xend = end), y = -9.25, yend = -9.25, color = '#ffbe0b', linewidth = 1) +
    geom_segment(data = ihs_wins, aes(x = start, xend = end), y = -9.5, yend = -9.5, color = '#8338ec', linewidth = 1) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 200000), labels = formatter1e6) +
    scale_y_continuous(breaks = seq(-9, 3, 2)) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
ggsave(filename = snakemake@output[["Chr08_Pall_rur_xpnsl"]], plot = manhat_plot, 
       height = 6, width = 8, device = "pdf", dpi = 600, units = "in")

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Rural') %>% 
    filter(Chr == 'Chr07_Occ') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_rural_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- xpnsl_persite %>%
    filter(Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

xpnsl_not_outlier <- region_raw_df %>% filter(crit == 0)
urban_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl < 0)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}

fst_outliers <- gt_fst_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
fst_wins <- bt.intersect(fst_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
nsl_outliers <- nsl_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
nsl_wins <- bt.intersect(nsl_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
ihs_outliers <- ihs_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
ihs_wins <- bt.intersect(ihs_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')

manhat_plot <- ggplot(xpnsl_not_outlier, aes(x = pos, y = normxpnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#003876', fill = '#003876',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#007243', fill = '#007243',
               aes(x = pos, y = normxpnsl)) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 07_Occ (position in Mbp)') + ylab('Normalized XP-nSL') +
    coord_cartesian(ylim = c(-6.5, 3.5)) +
    geom_segment(data = fst_wins, aes(x = start, xend = end), y = -6, yend = -6, color = '#d62828', linewidth = 1) +
    geom_segment(data = nsl_wins, aes(x = start, xend = end), y = -6.25, yend = -6.25, color = '#ffbe0b', linewidth = 1) +
    geom_segment(data = ihs_wins, aes(x = start, xend = end), y = -6.5, yend = -6.5, color = '#8338ec', linewidth = 1) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 200000), labels = formatter1e6) +
    scale_y_continuous(breaks = seq(-6, 4, 2)) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
ggsave(filename = snakemake@output[["Chr07_Occ_rur_xpnsl"]], plot = manhat_plot, 
       height = 6, width = 8, device = "pdf", dpi = 600, units = "in")

#### Chr05_Occ

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Rural') %>% 
    filter(Chr == 'Chr05_Occ') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_rural_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- xpnsl_persite %>%
    filter(Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

xpnsl_not_outlier <- region_raw_df %>% filter(crit == 0)
urban_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl < 0)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}

manhat_plot <- ggplot(xpnsl_not_outlier, aes(x = pos, y = normxpnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#003876', fill = '#003876',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#007243', fill = '#007243',
               aes(x = pos, y = normxpnsl)) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 05_Occ (position in Mbp)') + ylab('Normalized XP-nSL') +
    coord_cartesian(ylim = c(-5.5, 3.5)) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 200000), labels = formatter1e6) +
    scale_y_continuous(breaks = seq(-4, 4, 2)) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
ggsave(filename = snakemake@output[["Chr05_Occ_rur_xpnsl"]], plot = manhat_plot, 
       height = 6, width = 8, device = "pdf", dpi = 600, units = "in")

#### Chr04_Occ

In [ ]:
region_df <- xpnsl_top10_urban_rural %>% 
    filter(direction == 'Rural') %>% 
    filter(Chr == 'Chr04_Occ') %>%
    dplyr::select(Chr, start, end)

chrom <- region_df %>% pull(Chr) %>% unique()
min_pos <- region_df %>% pull(start) %>% min()
max_pos <- region_df %>% pull(end) %>% max()
buffer <- 500000

all_outlier_regions <- xpnsl_rural_outliers %>% 
    filter(Chr == chrom & start >= min_pos - buffer & end <= max_pos + buffer) %>% 
    dplyr::select(Chr, start, end)

region_raw_df <- xpnsl_persite %>%
    filter(Chr == chrom & pos >= min_pos - buffer & pos <= max_pos + buffer)

xpnsl_not_outlier <- region_raw_df %>% filter(crit == 0)
urban_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl > 0)
rural_xpnsl_outliers <- region_raw_df %>% filter(crit == 1 & normxpnsl < 0)

formatter1e6 <- function(x){ 
    x <- x / 1e6
    return(sprintf(x, fmt = '%#.2f'))
}
fst_outliers <- gt_fst_df_filt %>% filter(is_outlier == 1) %>% dplyr::select(Chr, start, end)
fst_wins <- bt.intersect(fst_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
nsl_outliers <- nsl_df_filt %>% filter(is_outlier == 1 & habitat == "Urban") %>% dplyr::select(Chr, start, end)
nsl_wins <- bt.intersect(nsl_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')
ihs_outliers <- ihs_df_filt %>% filter(is_outlier == 1 & habitat == "Urban") %>% dplyr::select(Chr, start, end)
ihs_wins <- bt.intersect(ihs_outliers, all_outlier_regions, wa = T) %>% 
    rename('Chr' = 'V1', 'start' = 'V2', 'end' = 'V3')

manhat_plot <- ggplot(xpnsl_not_outlier, aes(x = pos, y = normxpnsl)) +
    geom_rect(data=region_df, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_rect(data=all_outlier_regions, aes(xmin=start, xmax=end), ymin=-Inf, ymax=Inf, fill="black", alpha=0.2, inherit.aes = F) +
    geom_point(data = xpnsl_not_outlier, shape = 21, alpha = 0.3, size = 1, , fill = "black", color = "black") +
    geom_point(data = urban_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#003876', fill = '#003876',
               aes(x = pos, y = normxpnsl)) +
    geom_point(data = rural_xpnsl_outliers, shape = 21, alpha = 0.3, size = 1, color = '#007243', fill = '#007243',
               aes(x = pos, y = normxpnsl)) +
    geom_hline(yintercept = 2, color = "grey40", linetype = "dashed") +
    geom_hline(yintercept = -2, color = "grey40", linetype = "dashed") +
    xlab('Chromosome 05_Occ (position in Mbp)') + ylab('Normalized XP-nSL') +
    coord_cartesian(ylim = c(-6.5, 3.5)) +
    scale_x_continuous(breaks = seq(min_pos - buffer, max_pos + buffer, 200000), labels = formatter1e6) +
    geom_segment(data = fst_wins, aes(x = start, xend = end), y = -6, yend = -6, color = '#d62828', linewidth = 1) +
    geom_segment(data = nsl_wins, aes(x = start, xend = end), y = -6.25, yend = -6.25, color = '#ffbe0b', linewidth = 1) +
    geom_segment(data = ihs_wins, aes(x = start, xend = end), y = -6.5, yend = -6.5, color = '#8338ec', linewidth = 1) +
    scale_y_continuous(breaks = seq(-6, 4, 2)) +
    theme_classic() +
    theme(
        panel.border = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        axis.text = element_text(size=16),
        axis.title = element_text(size=20),
      )

options(repr.plot.width = 8, repr.plot.height = 6)
manhat_plot
ggsave(filename = snakemake@output[["Chr04_Occ_rur_xpnsl"]], plot = manhat_plot, 
       height = 6, width = 8, device = "pdf", dpi = 600, units = "in")